In [1]:
import pandas as pd
import sqlite3


In [2]:
df = pd.read_csv("C:/Users/harsh_m/OneDrive/Desktop/customer-analysis/data/processed/cleaned_telco_churn.csv")
df.head()


customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges  Churn  
0           Electronic check          29.85         29.85      0  
1               Mailed check          56.95       1889.50      0  
2               Mailed check          53.85        108.15      1  
3  Bank transfer (automatic)          42.30       1840.75      0  
4           Electronic check          70.70        151.65      1  

[5 rows x 21 columns]

In [3]:
conn = sqlite3.connect("churn.db")


In [4]:
df.to_sql("customers", conn, if_exists="replace", index=False)


7043

In [5]:
pd.read_sql("SELECT * FROM customers LIMIT 5", conn)


customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges  Churn  
0           Electronic check          29.85         29.85      0  
1               Mailed check          56.95       1889.50      0  
2               Mailed check          53.85        108.15      1  
3  Bank transfer (automatic)          42.30       1840.75      0  
4           Electronic check          70.70        151.65      1  

[5 rows x 21 columns]

In [6]:
query = """
SELECT Contract, COUNT(*) AS total_customers,
SUM(Churn) AS churned_customers
FROM customers
GROUP BY Contract;
"""
pd.read_sql(query, conn)


Contract  total_customers  churned_customers
0  Month-to-month             3875               1655
1        One year             1473                166
2        Two year             1695                 48

In [8]:

query = """
SELECT Contract,
COUNT(*) AS total,
SUM(Churn) AS churned,
ROUND(100.0 * SUM(Churn)/COUNT(*), 2) AS churn_rate
FROM customers
GROUP BY Contract;

"""
pd.read_sql(query, conn)

Contract  total  churned  churn_rate
0  Month-to-month   3875     1655       42.71
1        One year   1473      166       11.27
2        Two year   1695       48        2.83

In [9]:

query = """
SELECT PaymentMethod,
COUNT(*) AS total,
SUM(Churn) AS churned
FROM customers
GROUP BY PaymentMethod;


"""
pd.read_sql(query, conn)

PaymentMethod  total  churned
0  Bank transfer (automatic)   1544      258
1    Credit card (automatic)   1522      232
2           Electronic check   2365     1071
3               Mailed check   1612      308

In [13]:
query = """
SELECT Churn, AVG(tenure) AS avg_tenure
FROM customers
GROUP BY Churn;



"""
pd.read_sql(query, conn)

Churn  avg_tenure
0      0   37.569965
1      1   17.979133

In [10]:

query = """

SELECT ROUND(SUM(MonthlyCharges),2) AS monthly_revenue_lost
FROM customers
WHERE Churn = 1;



"""
pd.read_sql(query, conn)

monthly_revenue_lost
0             139130.85

monthly_revenue_lost
0             139130.85

In [12]:
query = """

SELECT customerID, tenure, MonthlyCharges
FROM customers
WHERE tenure < 12 AND MonthlyCharges > 70;




"""
pd.read_sql(query, conn)

customerID  tenure  MonthlyCharges
0    9237-HQITU       2           70.70
1    9305-CDSKC       8           99.65
2    4929-XIHVW       2           95.50
3    8168-UQWWF      11           97.85
4    7760-OYPDY       2           80.65
..          ...     ...             ...
821  1980-KXVPM       3           75.05
822  0723-DRCLG       1           74.45
823  1122-JWTJW       1           70.65
824  6894-LFHLY       1           75.75
825  8361-LTMKD       4           74.40

[826 rows x 3 columns]